In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## **Importing Libraries**

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression, Ridge, Lasso 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

## **Reading desired dataset**

In [ ]:
df = pd.read_csv("/kaggle/input/used-car-dataset-ford-and-mercedes/audi.csv")
df.head()

In [ ]:
#Checking for null values 
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
#Checking counts of model columns
df['model'].value_counts()

In [ ]:
#Changing year column to car age 
df["car_age"] = 2020 - df["year"]
df = df.drop(columns = ["year"])
df.head()

In [ ]:
#Converting categorical variable into numerical form using label encoder.
#Other option includes One - hot Encoder 
le = LabelEncoder()
df["le_transmission"] = le.fit_transform(df["transmission"])
df["le_fuelType"] = le.fit_transform(df["fuelType"])
df.head(10)

In [ ]:
#Droping unnecessory columns 
df = df.drop(columns = ["transmission", "fuelType"])
df.head()

In [ ]:
# Convert categorical variable ("model") into dummy variable
df_encoded = pd.get_dummies(df,columns=['model'])
df_encoded.head()

In [ ]:
#Splitting the data into training and testing
#It can be also split in three ways training, validation and testing sets during on your preference
X = df_encoded.drop(['price'],axis=1)
y = df_encoded['price']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## **Linear regression**

In [ ]:
linreg = LinearRegression().fit(X_train, y_train)

print('linear model coeff (w): {}'
     .format(linreg.coef_))
print('linear model intercept (b): {:.3f}'
     .format(linreg.intercept_))
print('R-squared score (training): {:.3f}'
     .format(linreg.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linreg.score(X_test, y_test)))

## **Ridge regression**

In [ ]:
linridge = Ridge(alpha=20.0).fit(X_train, y_train)

print('ridge regression linear model intercept: {}'
     .format(linridge.intercept_))
print('ridge regression linear model coeff:\n{}'
     .format(linridge.coef_))
print('R-squared score (training): {:.3f}'
     .format(linridge.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linridge.score(X_test, y_test)))
print('Number of non-zero features: {}'
     .format(np.sum(linridge.coef_ != 0)))

In [ ]:
linridge = Ridge(alpha=20.0).fit(X_train_scaled, y_train)

print('ridge regression linear model intercept: {}'
     .format(linridge.intercept_))
print('ridge regression linear model coeff:\n{}'
     .format(linridge.coef_))
print('R-squared score (training): {:.3f}'
     .format(linridge.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linridge.score(X_test_scaled, y_test)))
print('Number of non-zero features: {}'
     .format(np.sum(linridge.coef_ != 0)))

In [ ]:
print('Ridge regression: effect of alpha regularization parameter\n')
for this_alpha in [1, 10, 20, 50]:
    linridge = Ridge(alpha = this_alpha).fit(X_train_scaled, y_train)
    r2_train = linridge.score(X_train_scaled, y_train)
    r2_test = linridge.score(X_test_scaled, y_test)
    num_coeff_bigger = np.sum(abs(linridge.coef_) > 1.0)
    print('Alpha = {:.2f}\nnum abs(coeff) > 1.0: {}, \
r-squared training: {:.2f}, r-squared test: {:.2f}\n'
         .format(this_alpha, num_coeff_bigger, r2_train, r2_test))

## **Lasso Regression**



In [ ]:
linlasso = Lasso(alpha=2.0, max_iter = 10000).fit(X_train_scaled, y_train)

print('lasso regression linear model intercept: {}'
     .format(linlasso.intercept_))
print('lasso regression linear model coeff:\n{}'
     .format(linlasso.coef_))
print('Non-zero features: {}'
     .format(np.sum(linlasso.coef_ != 0)))
print('R-squared score (training): {:.3f}'
     .format(linlasso.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}\n'
     .format(linlasso.score(X_test_scaled, y_test)))

## **Support Vector Regression[](http://)**

In [ ]:
from sklearn.svm import SVR
pipeline_svr=Pipeline([('scalar1',StandardScaler()),
                     ('pca1',PCA(n_components=2)),
                     ('lr_classifier',SVR(kernel='linear'))])
pipeline_svr.fit(X_train, y_train)
pipeline_svr.score(X_test,y_test)

## **Decision Tree Regression**

In [ ]:
dt = DecisionTreeRegressor(random_state=0)

#test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 0 )

#standard scaler (fit transform on train, fit only on test)
sc = StandardScaler()
X_train = sc.fit_transform(X_train.astype(np.float))
X_test= sc.transform(X_test.astype(np.float))


#fit model
dt = dt.fit(X_train,y_train.values.ravel())
y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

#print score
print('dt train score %.3f, dt test score: %.3f' % (
dt.score(X_train,y_train),
dt.score(X_test, y_test)))

## **Random Forest Regression**


In [ ]:
forest = RandomForestRegressor(n_estimators = 100,
                              criterion = 'mse',
                              random_state = 1,
                              n_jobs = -1)
#test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 0 )

#standard scaler (fit transform on train, fit only on test)
sc = StandardScaler()
X_train = sc.fit_transform(X_train.astype(np.float))
X_test= sc.transform(X_test.astype(np.float))

#fit model
forest.fit(X_train,y_train.values.ravel())
y_train_pred = forest.predict(X_train)
y_test_pred = forest.predict(X_test)

#print score
print('forest train score %.3f, forest test score: %.3f' % (
forest.score(X_train,y_train),
forest.score(X_test, y_test)))

## **Gradient Boosting Regression**

In [ ]:
GB=GradientBoostingRegressor(random_state=0)
#test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 0 )

#standard scaler (fit transform on train, fit only on test)
sc = StandardScaler()
X_train = sc.fit_transform(X_train.astype(np.float))
X_test= sc.transform(X_test.astype(np.float))


#fit model
gb = GB.fit(X_train,y_train.values.ravel())
y_train_pred = gb.predict(X_train)
y_test_pred = gb.predict(X_test)

#print score
print('GB train score %.3f, GB test score: %.3f' % (
gb.score(X_train,y_train),
gb.score(X_test, y_test)))

## **XGB Regression** 

In [ ]:
XGB=XGBRegressor(random_state=0)
#test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 0 )

#standard scaler (fit transform on train, fit only on test)
sc = StandardScaler()
X_train = sc.fit_transform(X_train.astype(np.float))
X_test= sc.transform(X_test.astype(np.float))


#fit model
xgb = XGB.fit(X_train,y_train.values.ravel())
y_train_pred = gb.predict(X_train)
y_test_pred = gb.predict(X_test)

#print score
print('XGB train score %.3f, XGB test score: %.3f' % (
xgb.score(X_train,y_train),
xgb.score(X_test, y_test)))

## **AdaBoost Regression**

In [ ]:
ABR=AdaBoostRegressor(random_state=0)
#test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 0 )

#standard scaler (fit transform on train, fit only on test)
sc = StandardScaler()
X_train = sc.fit_transform(X_train.astype(np.float))
X_test= sc.transform(X_test.astype(np.float))


#fit model
abr = ABR.fit(X_train,y_train.values.ravel())
y_train_pred = abr.predict(X_train)
y_test_pred = abr.predict(X_test)

#print score
print('ABR train score %.3f, ABR test score: %.3f' % (
abr.score(X_train,y_train),
abr.score(X_test,y_test)))

**We notice during the comparion of various regressions score
* Random Forest Regression : 95.6
* **eXtreme Gradient Boost Regression : 95.3

Hyperparameter tuning can be performed on both of these regressor.
**

**Hyperparameter Tuning**

In [ ]:
def print_best_params(gd_model):
    param_dict = gd_model.best_estimator_.get_params()
    model_str = str(gd_model.estimator).split('(')[0]
    print("\n*** {} Best Parameters ***".format(model_str))
    for k in param_dict:
        print("{}: {}".format(k, param_dict[k]))
    print()

In [ ]:
param_grid_rf = dict(n_estimators=[20],
                     max_depth=np.arange(1, 15, 2),
                     min_samples_split=[2],
                     min_samples_leaf= np.arange(1, 15, 2, int),
                     bootstrap=[True, False],
                     oob_score=[False, ])


forest = GridSearchCV(RandomForestRegressor(random_state=0), param_grid=param_grid_rf, cv=10, verbose=3)

#fit model
forest.fit(X_train,y_train.values.ravel())


#print score
print('\n\nforest train score %.3f, forest test score: %.3f' % (
forest.score(X_train,y_train),
forest.score(X_test, y_test)))

print_best_params(forest)

In [ ]:
xgb1 = XGBRegressor()
parameters = {'nthread':[2], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [200]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 10,
                        n_jobs = -1,
                        verbose=True)

xgb_grid.fit(X_train,
         y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

## **XG Boost Regression**
96.17717084396699
* Parameters: 
{'colsample_bytree': 0.7, 'learning_rate': 0.07, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 200, 'nthread': 2, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}